In [1]:
import numpy as np
import timor
import argparse
import pygad
import os
from util import *
from timor.Module import *
from timor.utilities.visualization import animation
# import matplotlib.pyplot as plt
# import itertools
# from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
from timor import ModuleAssembly, ModulesDB
from timor.Bodies import Body, Connector, Gender
from timor.configuration_search.GA import GA
from timor.utilities.visualization import MeshcatVisualizer, clear_visualizer
from timor.utilities.dtypes import Lexicographic
from timor.utilities.transformation import Transformation
from timor.utilities.spatial import rotX, rotY, rotZ
from timor.Module import AtomicModule, ModulesDB, ModuleHeader
from timor.Joints import Joint
from timor.Geometry import Box, ComposedGeometry, Cylinder, Sphere, Mesh

from reachability import Reachability
from generate_module import create_i_links, create_eef, create_revolute_joint

2025-02-21 15:10:38,447 Timor INFO Loading custom configurations from /home/mscsim/.config/timor.config
2025-02-21 15:10:38,984 Timor INFO Getting robot modrob-gen2.
2025-02-21 15:10:38,986 Timor WARNING Could not fetch robot modrob-gen2 due to [Errno 17] File exists: '/home/mscsim/anaconda3/envs/timor-env/lib/python3.10/site-packages/cache/robots/modrob-gen2'.


In [2]:
how_many_times_to_split_angle_range = 30
world_resolution = 0.01
world_dimension = [1.00, 1.00, 1.00]
num_threads = 5
our_hyperparameters = {
    'population_size': 10,
    'num_generations': 50,
    'num_genes': 6,
    'save_solutions_dir': None
}

In [3]:
r_4310_base = create_revolute_joint("assets/Assem_4310_BASE/Assem_4310_BASE/urdf/Assem_4310_BASE.urdf")
r_4305_joint = create_revolute_joint("assets/Assem_4305_JOINT/Assem_4305_JOINT/urdf/Assem_4305_JOINT.urdf")
r_4310_joint = create_revolute_joint("assets/Assem_4310_JOINT/Assem_4310_JOINT/urdf/Assem_4310_JOINT.urdf")

# Links

baseto4310_links = create_i_links(rod_name="baseto4310")
r4310to4305_links = create_i_links(rod_name="r4310to4305")
r4310to4310_links = create_i_links(rod_name="r4310to4310")

# eef = create_eef()

# Create database
db = ModulesDB()
db.add(r_4310_joint)
db.add(r_4310_base)
db.add(r_4305_joint)
db = db.union(baseto4310_links)
db = db.union(r4310to4305_links)
db = db.union(r4310to4310_links)
# db = db.union(eef)
viz = db.debug_visualization()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [4]:
modules = ('base_rev_joint','baseto4310-0.3','motor4310_rev_joint')
B = ModuleAssembly.from_serial_modules(db, modules)
robot = B.to_pin_robot()
print(robot.has_self_collision())
viz = robot.visualize()

False
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
